[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zlik/llama-stack/blob/main/docs/getting_started/03_AgentEvaluations.ipynb)

## 3. Llama Stack Agent Evaluations


#### 3.1. Online Evaluation Dataset Collection

- Llama Stack allows you to query each steps of the agents execution in your application. 
- In this example, we will show how to 
    1. build an Agent with Llama Stack
    2. Query the agent's session, turns, and steps
    3. Evaluate the results

##### 3.1.1. Building a Search Agent

First, let's build an agent that have access to a search tool with Llama Stack, and use it to run some user queries. 

In [22]:
from llama_stack_client import Agent, AgentEventLogger

agent = Agent(
    client, 
    model="meta-llama/Llama-3.3-70B-Instruct",
    instructions="You are a helpful assistant. Use web_search tool to answer the questions.",
    tools=["builtin::websearch"],
)
user_prompts = [
    "Which teams played in the NBA western conference finals of 2024. Search the web for the answer.",
    "In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.",
    "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
]

session_id = agent.create_session(uuid.uuid4().hex)

for prompt in user_prompts:
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
    )

    for log in AgentEventLogger().log(response):
        log.print()


inference> brave_search.call(query="NBA Western Conference Finals 2024 teams")
tool_execution> Tool:brave_search Args:{'query': 'NBA Western Conference Finals 2024 teams'}
tool_execution> Tool:brave_search Response:{"query": "NBA Western Conference Finals 2024 teams", "top_k": [{"title": "2024 NBA Western Conference Finals - Basketball-Reference.com", "url": "https://www.basketball-reference.com/playoffs/2024-nba-western-conference-finals-mavericks-vs-timberwolves.html", "content": "2024 NBA Playoffs Dallas Mavericks vs. Dallas Mavericks vs. Dallas Mavericks vs. 5 Dallas Mavericks (4-1) vs. 7   Derrick Jones Jr. 2024 NBA Playoffs Dallas Mavericks vs. Dallas Mavericks vs. Dallas Mavericks vs. College Tools: Player Season Finder, Player Game Finder, Team Season Finder, Team Game Finder Players, Teams, Seasons, Leaders, Awards ... Players, Teams, Seasons, Leaders, Awards ... Players, Teams, Seasons, Leaders, Awards, All-Star Games, Executives ... Players, Teams, Seasons, Leaders, Awards .

##### 3.1.2 Query Agent Execution Steps

Now, let's look deeper into the agent's execution steps and see if how well our agent performs. As a sanity check, we will first check if all user prompts is followed by a tool call to `brave_search`.

In [ ]:
# query the agents session
from rich.pretty import pprint

session_response = client.agents.session.retrieve(
    session_id=session_id,
    agent_id=agent.agent_id,
)

pprint(session_response.turns)

In [24]:
num_tool_call = 0
for turn in session_response.turns:
    for step in turn.steps:
        if step.step_type == "tool_execution" and step.tool_calls[0].tool_name == "brave_search":
            num_tool_call += 1

print(f"{num_tool_call}/{len(session_response.turns)} user prompts are followed by a tool call to `brave_search`")

3/3 user prompts are followed by a tool call to `brave_search`


##### 3.1.3 Evaluate Agent Responses

Now, we want to evaluate the agent's responses to the user prompts. 

1. First, we will process the agent's execution history into a list of rows that can be used for evaluation.
2. Next, we will label the rows with the expected answer.
3. Finally, we will use the `/scoring` API to score the agent's responses.

In [25]:
eval_rows = []

expected_answers = [
    "Dallas Mavericks and the Minnesota Timberwolves",
    "Season 4, Episode 12",
    "King Cobra",
]

for i, turn in enumerate(session_response.turns):
    eval_rows.append(
        {
            "input_query": turn.input_messages[0].content,
            "generated_answer": turn.output_message.content,
            "expected_answer": expected_answers[i],
        }
    )

pprint(eval_rows)

scoring_params = {
    "basic::subset_of": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)

[
│   {
│   │   'input_query': 'Which teams played in the NBA western conference finals of 2024. Search the web for the answer.',
│   │   'generated_answer': 'The teams that played in the NBA Western Conference Finals of 2024 were the Dallas Mavericks and the Minnesota Timberwolves.',
│   │   'expected_answer': 'Dallas Mavericks and the Minnesota Timberwolves'
│   },
│   {
│   │   'input_query': 'In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.',
│   │   'generated_answer': 'Bill Cosby first appears in the episode "Trapped in the Closet" (Season 9, Episode 12) of South Park.',
│   │   'expected_answer': 'Season 4, Episode 12'
│   },
│   {
│   │   'input_query': "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
│   │   'generated_answer': 'Andrew Tate\'s kickboxing name is "King Cobra".',
│   │   'expected_answer': 'King Cobra'
│   }
]

ScoringScoreResponse(
│   results={
│   │   'basic::subset_of': ScoringResult(
│   │   │   aggregated_results={'accuracy': {'accuracy': 0.6666666666666666, 'num_correct': 2.0, 'num_total': 3}},
│   │   │   score_rows=[{'score': 1.0}, {'score': 0.0}, {'score': 1.0}]
│   │   )
│   }
)

##### 3.1.4 Query Telemetry & Evaluate

Another way to get the agent's execution history is to query the telemetry logs from the `/telemetry` API. The following example shows how to query the telemetry logs and post-process them to prepare data for evaluation.

In [26]:
# NBVAL_SKIP
print(f"Getting traces for session_id={session_id}")
import json

from rich.pretty import pprint

agent_logs = []

for span in client.telemetry.query_spans(
    attribute_filters=[
        {"key": "session_id", "op": "eq", "value": session_id},
    ],
    attributes_to_return=["input", "output"],
):
    if span.attributes["output"] != "no shields":
        agent_logs.append(span.attributes)

print("Here are examples of traces:")
pprint(agent_logs[:2])


Getting traces for session_id=d73d9aaa-65ac-4255-8153-9f5cbff6e01e
Here are examples of traces:


[
│   {
│   │   'input': '[{"role": "system", "content": "You are a helpful assistant. Use web_search tool to answer the questions."}, {"role": "user", "content": "Which teams played in the NBA western conference finals of 2024. Search the web for the answer.", "context": null}]',
│   │   'output': '{"content": "", "tool_calls": [{"call_id": "5f77ab69-72d9-4d51-b96c-bd4352ced54a", "tool_name": "brave_search", "arguments": {"query": "NBA Western Conference Finals 2024 teams"}, "arguments_json": "{\\"query\\": \\"NBA Western Conference Finals 2024 teams\\"}"}]}'
│   },
│   {
│   │   'input': '{"role":"assistant","content":"","stop_reason":"end_of_turn","tool_calls":[{"call_id":"5f77ab69-72d9-4d51-b96c-bd4352ced54a","tool_name":"brave_search","arguments":{"query":"NBA Western Conference Finals 2024 teams"},"arguments_json":"{\\"query\\": \\"NBA Western Conference Finals 2024 teams\\"}"}]}',
│   │   'output': '{"role":"tool","call_id":"5f77ab69-72d9-4d51-b96c-bd4352ced54a","content":"{\\"query\\": \\"NBA Western Conference Finals 2024 teams\\", \\"top_k\\": [{\\"title\\": \\"2024 NBA Western Conference Finals - Basketball-Reference.com\\", \\"url\\": \\"https://www.basketball-reference.com/playoffs/2024-nba-western-conference-finals-mavericks-vs-timberwolves.html\\", \\"content\\": \\"2024 NBA Playoffs Dallas Mavericks vs. Dallas Mavericks vs. Dallas Mavericks vs. 5 Dallas Mavericks (4-1) vs. 7   Derrick Jones Jr. 2024 NBA Playoffs Dallas Mavericks vs. Dallas Mavericks vs. Dallas Mavericks vs. College Tools: Player Season Finder, Player Game Finder, Team Season Finder, Team Game Finder Players, Teams, Seasons, Leaders, Awards ... Players, Teams, Seasons, Leaders, Awards ... Players, Teams, Seasons, Leaders, Awards, All-Star Games, Executives ... Players, Teams, Seasons, Leaders, Awards ... Subscribe to Stathead Basketball: Get your first month FREE The SPORTS REFERENCE, STATHEAD, IMMACULATE GRID, and IMMACULATE FOOTY trademarks are owned exclusively by Sports Reference LLC. Sports\\\\u00a0Reference\\\\u202f\\\\u00ae Baseball Football (college) Basketball (college) Hockey F\\\\u00fatbol Blog Stathead\\\\u202f\\\\u00ae Immaculate Grid\\\\u202f\\\\u00ae\\", \\"score\\": 0.89030397, \\"raw_content\\": null}, {\\"title\\": \\"NBA Standings - 2024-25 season - ESPN\\", \\"url\\": \\"https://www.espn.com/nba/standings\\", \\"content\\": \\"NBA Standings - 2024-25 season - ESPN Skip to main contentSkip to navigation ESPN NFL NBA NCAAF NHL NCAAM NCAAW Soccer More Sports Watch Fantasy NBA Home Scores Schedule Standings Stats Teams Odds Where To Watch All-Star Game Fantasy More NBA Standings 2024-25 Standings Expanded Vs. Division NBA Cup LeagueConferenceDivision Eastern Conference | | | --- | | 1CLECleveland Cavaliers | | 2BOSBoston Celtics | | 3NYNew York Knicks | | 4INDIndiana Pacers | | 5MILMilwaukee Bucks | | 6DETDetroit Pistons | | 7MIAMiami Heat | | 8ORLOrlando Magic | | 9ATLAtlanta Hawks | | 10CHIChicago Bulls | | PHIPhiladelphia 76ers | | BKNBrooklyn Nets | | TORToronto Raptors | | CHACharlotte Hornets | | WSHWashington Wizards | | W | L | PCT | GB | HOME | AWAY | DIV | CONF | PPG | OPP PPG | DIFF | STRK | L10 | | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | | 42 | 10 | .808 | - | 24-4 | 18-6 | 9-1 | 28-7 | 122.4 | 112.1 | +10.3 | W2 | 6-4 | | 36 | 16 | .692 | 6 | 16-10 | 20-6 | 6-2 | 26-9 | 117.3 | 108.8 | +8.5 | L1 | 7-3 | | 34 | 17 | .667 | 7.5 | 18-9 | 16-8 | 9-1 | 23-10 | 117.9 | 111.4 | +6.5 | W2 | 8-2 | | 29 | 21 | .580 | 12 | 14-7 | 14-13 | 6-4 | 17-15 | 115.7 | 114.9 | +0.8 | W1 | 7-3 | | 27 | 23 | .540 | 14 | 16-8 | 10-15 | 6-5 | 22-16 | 114.2 | 112.6 | +1.6 | L1 | 4-6 | | 26 | 26 | .500 | 16 | 13-13 | 13-13 | 2-9 | 18-20 | 113.0 | 113.8 | -0.8 | W1 | 5-5 | | 25 | 25 | .500 | 16 | 12-10 | 12-15 | 5-3 | 14-15 | 110.5 | 110.6 | -0.1 | L1 | 5-5 | | 25 | 28 | .472 | 17.5 | 15-9 | 10-19 | 5-2 | 20-15 | 103.8 | 105.6 | -1.8 | L1 | 2-8 | | 24 | 28 | .462 | 18 | 12-12 | 12-15 | 4-2 | 17-13 | 116.1 |

- Now, we want to run evaluation to assert that our search agent succesfully calls brave_search from online traces.
- We will first post-process the agent's telemetry logs and run evaluation.

In [27]:
# NBVAL_SKIP
# post-process telemetry spance and prepare data for eval
# in this case, we want to assert that all user prompts is followed by a tool call
import ast
import json

eval_rows = []

for log in agent_logs:
    input = json.loads(log["input"])
    if isinstance(input, list):
        input = input[-1]
    if input["role"] == "user":
        eval_rows.append(
            {
                "input_query": input["content"],
                "generated_answer":  log["output"],
                # check if generated_answer uses tools brave_search
                "expected_answer": "brave_search",
            },
        )

# pprint(eval_rows)
scoring_params = {
    "basic::subset_of": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)


ScoringScoreResponse(
│   results={
│   │   'basic::subset_of': ScoringResult(
│   │   │   aggregated_results={'accuracy': {'accuracy': 1.0, 'num_correct': 3.0, 'num_total': 3}},
│   │   │   score_rows=[{'score': 1.0}, {'score': 1.0}, {'score': 1.0}]
│   │   )
│   }
)

#### 3.2. Agentic Application Dataset Scoring
- Llama Stack offers a library of scoring functions and the `/scoring` API, allowing you to run evaluations on your pre-annotated AI application datasets.

- In this example, we will work with an example RAG dataset you have built previously, label with an annotation, and use LLM-As-Judge with custom judge prompt for scoring. Please checkout our [Llama Stack Playground](https://llama-stack.readthedocs.io/en/latest/playground/index.html) for an interactive interface to upload datasets and run scorings.

In [28]:
import rich
from rich.pretty import pprint

# could even use larger models like 405B
judge_model_id = "meta-llama/Llama-3.3-70B-Instruct"

JUDGE_PROMPT = """
Given a QUESTION and GENERATED_RESPONSE and EXPECTED_RESPONSE.

Compare the factual content of the GENERATED_RESPONSE with the EXPECTED_RESPONSE. Ignore any differences in style, grammar, or punctuation.
  The GENERATED_RESPONSE may either be a subset or superset of the EXPECTED_RESPONSE, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
  (A) The GENERATED_RESPONSE is a subset of the EXPECTED_RESPONSE and is fully consistent with it.
  (B) The GENERATED_RESPONSE is a superset of the EXPECTED_RESPONSE and is fully consistent with it.
  (C) The GENERATED_RESPONSE contains all the same details as the EXPECTED_RESPONSE.
  (D) There is a disagreement between the GENERATED_RESPONSE and the EXPECTED_RESPONSE.
  (E) The answers differ, but these differences don't matter from the perspective of factuality.

Give your answer in the format "Answer: One of ABCDE, Explanation: ".

Your actual task:

QUESTION: {input_query}
GENERATED_RESPONSE: {generated_answer}
EXPECTED_RESPONSE: {expected_answer}
"""

input_query = (
    "What are the top 5 topics that were explained? Only list succinct bullet points."
)
generated_answer = """
Here are the top 5 topics that were explained in the documentation for Torchtune:

* What is LoRA and how does it work?
* Fine-tuning with LoRA: memory savings and parameter-efficient finetuning
* Running a LoRA finetune with Torchtune: overview and recipe
* Experimenting with different LoRA configurations: rank, alpha, and attention modules
* LoRA finetuning
"""
expected_answer = """LoRA"""

rows = [
    {
        "input_query": input_query,
        "generated_answer": generated_answer,
        "expected_answer": expected_answer,
    },
]

scoring_params = {
    "llm-as-judge::base": {
        "judge_model": judge_model_id,
        "prompt_template": JUDGE_PROMPT,
        "type": "llm_as_judge",
        "judge_score_regexes": ["Answer: (A|B|C|D|E)"],
    },
    "basic::subset_of": None,
}

response = client.scoring.score(input_rows=rows, scoring_functions=scoring_params)
pprint(response)


ScoringScoreResponse(
│   results={
│   │   'llm-as-judge::base': ScoringResult(
│   │   │   aggregated_results={},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 'B',
│   │   │   │   │   'judge_feedback': 'Answer: B, Explanation: The GENERATED_RESPONSE is a superset of the EXPECTED_RESPONSE and is fully consistent with it. The EXPECTED_RESPONSE only mentions "LoRA", which is a topic that is extensively covered in the GENERATED_RESPONSE. The GENERATED_RESPONSE provides more specific and detailed topics related to LoRA, but it does not contradict the EXPECTED_RESPONSE.'
│   │   │   │   }
│   │   │   ]
│   │   ),
│   │   'basic::subset_of': ScoringResult(
│   │   │   aggregated_results={'accuracy': {'accuracy': 1.0, 'num_correct': 1.0, 'num_total': 1}},
│   │   │   score_rows=[{'score': 1.0}]
│   │   )
│   }
)